# Create windows lists for the existing set of landsat files

This notebook would be used after fetching landsat tiles.  It will create windows from the tiles, and filter them (to skip empty ones).  You can use the visualization tools to see where the windows are superimposed on masks of the corine data.  Note that landsat tiles can cover the same location at different times, so 'doubled' rectangles aren't necessarily a bug.

You have to move this notebook to the parent directory (or else add the parent directory to your python path) in order to use it.  Otherwise the multispectral imports below will fail.



In [1]:
import rasterio
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot
from pathlib import Path

In [3]:
import sys
sys.path.append('/home/firewise')
from multispectral import corine
from multispectral import coords
from multispectral import windows
from multispectral import tools
tools.set_figure_width(15)

In [8]:
alltiles = list(Path('/home/usgs/landsat5/').glob('*.tif'))

In [9]:
alltiles[:5]

In [10]:
wi = windows.prefilter(alltiles, corine.corine_filter)

In [11]:
wl = list(wi)

In [12]:
len(wl)

In [13]:
corine._tile_counts

In [14]:
windows.to_file(wl,'all_windows_v2.csv')

In [3]:
wl = windows.from_file('all_windows.csv')

In [15]:
cdats = {}
for (ld,w) in wl:
    cdat = corine.fetch_corine(ld.crs)
    if cdat not in cdats:
        cdats[cdat] = []
    cdats[cdat].append((ld,w))

In [16]:
for cdat in cdats:
    print("{}: {}".format(cdat.name, len(cdats[cdat])))

In [17]:
def show_window_list(wl,cdat):
    """Show the window list superimposed on the specified dataset (generally a corine tile)."""
    rs = [coords.geo_to_pixel(cdat, coords.pixel_to_geo(x,y)).flatten() for (x,y) in wl ]
    tools.show_band(cdat,1,showrect=rs)

In [18]:
for cdat, wl in cdats.items():
    show_window_list( wl, cdat )